# Install required modules

In [ ]:
%%capture
!pip install bertopic datasets accelerate bitsandbytes xformers adjustText
# datasets

In [ ]:
from datetime import timedelta
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
#from pandas_summary import DataFrameSummary
import seaborn as sns
import numpy as np
#import vertica_python
import warnings
import datetime as dt

# **Data**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the data
fname = "drive//MyDrive//Diploma//second_approach_2705.xlsx"
df = pd.read_excel(fname)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41562 entries, 0 to 41561
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              41562 non-null  int64         
 1   req_id                  41562 non-null  object        
 2   req_header              41562 non-null  object        
 3   req_details             41470 non-null  object        
 4   reg_date                41562 non-null  datetime64[ns]
 5   fr_mod_lem              41562 non-null  object        
 6   fr_abbr_with_punct      41562 non-null  object        
 7   fr_mod_lem_swa          41562 non-null  object        
 8   fr_abbr_with_punct_swa  41562 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 2.9+ MB


In [ ]:
#df.dropna(inplace=True)

In [17]:
#df.head(1)

In [ ]:
df.columns

Index(['Unnamed: 0', 'req_id', 'req_header', 'req_details', 'reg_date',
       'fr_mod_lem', 'fr_abbr_with_punct', 'fr_mod_lem_swa',
       'fr_abbr_with_punct_swa'],
      dtype='object')

In [ ]:
#df.columns = ['unnamed','id','title','info','date_reg','full_request_modified_lem','full_request_modified_lem_stop_words_applied','full_request_with_punct']

In [ ]:
df['date_reg'] = pd.to_datetime(df['reg_date'])

In [18]:
#df.head(1)

In [ ]:
df = df[['req_id', 'req_header', 'req_details', 'reg_date',
       'fr_mod_lem', 'fr_abbr_with_punct', 'fr_mod_lem_swa',
       'fr_abbr_with_punct_swa']]

In [ ]:
abstracts_lem = df["fr_mod_lem"]
abstracts_lem_swa = df["fr_mod_lem_swa"]
abstracts_with_punct = df['fr_abbr_with_punct']
abstracts_with_punct_swa = df['fr_abbr_with_punct_swa']
titles = df["req_header"]

In [ ]:
from collections import defaultdict

In [ ]:
check_words = ['ru', 'sbrf', 'tvlod', 'pplad', 'база', 'pangolin']
checks_frequency = defaultdict(int)
def checkword(text):
    text2list = text.split(' ')
    for token in text2list:
        if token in check_words:
            checks_frequency[token]+=1
checks_frequency = defaultdict(int)
checks_frequency
abstracts_with_punct_swa.apply(checkword)
checks_frequency

 **BERTopic**

We  need to perform two steps:
* Pre-calculating Embeddings
* Defining Sub-models

## **Preparing Embeddings**



In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model_msmarco = SentenceTransformer("DiTy/bi-encoder-russian-msmarco")
embedding_model_inkoz = SentenceTransformer("inkoziev/sbert_synonymy")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Create embeddings
embeddings_msmarco = embedding_model_msmarco.encode(abstracts_with_punct, show_progress_bar=True)


Batches:   0%|          | 0/1299 [00:00<?, ?it/s]

In [ ]:
embeddings_inkoz = embedding_model_inkoz.encode(abstracts_with_punct, show_progress_bar=True)

Batches:   0%|          | 0/1299 [00:00<?, ?it/s]

## **Sub-models**



In [ ]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
#from sklearn.cluster import HDBSCAN as SKHDBSCAN

umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

we are reducing the embeddings to 2-dimensions so that we can make visualization.

In [ ]:
# Pre-reduce embeddings for visualization purposes
#reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

### **Representation Models**

We create additional representations to view a topic from multiple angles.

Here, we use c-TF-IDF as our main representation and [KeyBERT](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#keybertinspired), [MMR](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#maximalmarginalrelevance)

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Llama 2
#llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
 #   "Llama2": llama2,
    "MMR": mmr,
}

# **Training**



# First launch of models (msmarco, inkoziev)
## We use no stop words!

### MsMarco model

In [ ]:
from bertopic import BERTopic

topic_model_msmarco = BERTopic(

  # Sub-models
  embedding_model = embedding_model_msmarco,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model_msmarco.fit_transform(abstracts_with_punct, embeddings_msmarco)

2024-05-28 06:44:53,498 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-28 06:46:19,149 - BERTopic - Dimensionality - Completed ✓
2024-05-28 06:46:19,152 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
2024-05-28 06:46:29,526 - BERTopic - Cluster - Completed ✓
2024-05-28 06:46:29,538 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-28 06:46:33,370 - BERTopic - Representation - Completed ✓


Now that we are done training our model, let's see what topics were generated:

In [19]:
# Show topics
#topic_model_msmarco.get_topic_info()

In [20]:
#topic_model_msmarco.visualize_barchart(top_n_topics = 6, n_words = 7)

In [ ]:
tp_df = topic_model_msmarco.get_topic_info()  # dataframe with topics
topics = pd.Series(topic_model_msmarco.topics_, index = df.index, name = 'topic_id')  # request - topic pairs
results = df.join(topics).merge(tp_df[['Topic','Representation','MMR']], left_on = 'topic_id', right_on = 'Topic')

In [ ]:
results.to_csv("drive//MyDrive//bertopic_requests_topic_msmarco_with_punct_no_stop_words.csv")

In [ ]:
tp_df.to_csv("drive//MyDrive//bertopic_topics_msmarco_with_punct_no_stop_words.csv")

In [1]:
#topic_model_msmarco.get_topic(0, full=True)

In [ ]:
path = 'drive//MyDrive//Diploma//'
fname = path+'topic_model_msmarco'
topic_model_msmarco.save(fname)

In [ ]:

fname

'drive//MyDrive//Diploma//topic_model_msmarco'

2024-05-28 06:51:19,915 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



### Inkoziev model

In [ ]:
from bertopic import BERTopic

topic_model_inkoz = BERTopic(

  # Sub-models
  embedding_model = embedding_model_inkoz,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model_inkoz.fit_transform(abstracts_with_punct, embeddings_inkoz)

2024-05-28 06:52:06,491 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-28 06:53:08,859 - BERTopic - Dimensionality - Completed ✓
2024-05-28 06:53:08,862 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

2024-05-28 06:53:17,655 - BERTopic - Cluster - Completed ✓
2024-05-28 06:53:17,666 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-28 06:53:20,653 - BERTopic - Representation - Completed ✓


In [21]:
# Show topics
#topic_model_inkoz.get_topic_info()

In [22]:
#topic_model_inkoz.visualize_barchart(top_n_topics = 7, n_words = 7)

In [ ]:
tp_df = topic_model_inkoz.get_topic_info()  # dataframe with topics
topics = pd.Series(topic_model_inkoz.topics_, index = df.index, name = 'topic_id')  # request - topic pairs
results = df.join(topics).merge(tp_df[['Topic','Representation','MMR']], left_on = 'topic_id', right_on = 'Topic')
results.to_csv("drive//MyDrive//bertopic_requests_topic_inkoz_no_stop_words.csv")
tp_df.to_csv("drive//MyDrive//bertopic_topics_inkozno_stop_words.csv")

In [ ]:
topic_model_inkoz.get_topic(0, full=True)

{'Main': [('на', 0.04696301013583564),
  ('данных', 0.044383266266043435),
  ('ru', 0.039913639502352355),
  ('sbrf', 0.03930469595521457),
  ('база', 0.03831155948163839),
  ('для', 0.02825744975708984),
  ('tvldd', 0.02795451497263748),
  ('pprb', 0.025181088878032304),
  ('запрос', 0.024701132461781662),
  ('as', 0.02455349443170881)],
 'KeyBERT': [('система', 0.6590998),
  ('учетная', 0.6076385),
  ('grant', 0.60679746),
  ('база', 0.5689281),
  ('тестирование', 0.5599239),
  ('search', 0.5563332),
  ('admin', 0.5513598),
  ('стенд', 0.5338431),
  ('элемент', 0.51411337),
  ('тестирования', 0.51380104)],
 'MMR': [('на', 0.04696301013583564),
  ('данных', 0.044383266266043435),
  ('ru', 0.039913639502352355),
  ('sbrf', 0.03930469595521457),
  ('база', 0.03831155948163839),
  ('для', 0.02825744975708984),
  ('tvldd', 0.02795451497263748),
  ('pprb', 0.025181088878032304),
  ('запрос', 0.024701132461781662),
  ('as', 0.02455349443170881)]}

In [ ]:
path = 'drive//MyDrive//Diploma//'
fname = path+'model_inkoz'
topic_model_msmarco.save(fname)

2024-05-28 06:58:26,376 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
titles

0        Недоступность системы/замедление в работе RLM
1                                    Нестандартный ЗНО
2                                    Нестандартный ЗНО
3                           Выдача прав на БД Pangolin
4                           Выдача прав на БД Pangolin
                             ...                      
41557                                       pg_profile
41558                                       pg_profile
41559                                       pg_profile
41560         ошибки в конфигурационных файлах постгре
41561                                Нестандартный ЗНО
Name: req_header, Length: 41562, dtype: object

# **Visualize**


In [2]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings_msmarco = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings_msmarco)

NameError: name 'UMAP' is not defined

In [3]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings_inkoz = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings_inkoz)

NameError: name 'UMAP' is not defined

In [ ]:
topic_model_inkoz.get_topic(0, )

In [ ]:
titles_ex = ['one','two','three','four','five','six','seven']

### Visualize MsMarco

In [ ]:
topic_model_msmarco.visualize_documents(titles, reduced_embeddings=reduced_embeddings_msmarco, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [ ]:
topic_model_inkoz.visualize_documents(titles, reduced_embeddings=reduced_embeddings_inkoz, hide_annotations=True, hide_document_hover=False, custom_labels=True)

# **Advanced Visualization**

 We can also create a static that might be a bit more informative.



In [ ]:
"""
import itertools
import pandas as pd

# Define colors for the visualization to iterate over
colors = itertools.cycle(['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe'])
#color_key = {str(topic): next(colors) for topic in set(topic_model.topics_) if topic != -1}
color_key = {topic: list(plt.cm.tab10(int(topic))) for topic in set(topic_model.topics_) if topic != -1}


# Prepare dataframe and ignore outliers
dft = pd.DataFrame({"x": reduced_embeddings[:, 0], "y": reduced_embeddings[:, 1], "Topic": [str(t) for t in topic_model.topics_]})
dft["Length"] = [len(doc) for doc in abstracts]
dft = dft.loc[dft.Topic != "-1"]
#df = df.loc[(df.y > -100) & (df.y < 100) & (df.x < 10) & (df.x > -10), :]
dft["Topic"] = dft["Topic"].astype(int)

# Get centroids of clusters
mean_df = dft.groupby("Topic").mean().reset_index()
mean_df.Topic = mean_df.Topic.astype(int)
mean_df = mean_df.sort_values("Topic")
"""

'\nimport itertools\nimport pandas as pd\n\n# Define colors for the visualization to iterate over\ncolors = itertools.cycle([\'#e6194b\', \'#3cb44b\', \'#ffe119\', \'#4363d8\', \'#f58231\', \'#911eb4\', \'#46f0f0\', \'#f032e6\', \'#bcf60c\', \'#fabebe\'])\n#color_key = {str(topic): next(colors) for topic in set(topic_model.topics_) if topic != -1}\ncolor_key = {topic: list(plt.cm.tab10(int(topic))) for topic in set(topic_model.topics_) if topic != -1}\n\n\n# Prepare dataframe and ignore outliers\ndft = pd.DataFrame({"x": reduced_embeddings[:, 0], "y": reduced_embeddings[:, 1], "Topic": [str(t) for t in topic_model.topics_]})\ndft["Length"] = [len(doc) for doc in abstracts]\ndft = dft.loc[dft.Topic != "-1"]\n#df = df.loc[(df.y > -100) & (df.y < 100) & (df.x < 10) & (df.x > -10), :]\ndft["Topic"] = dft["Topic"].astype(int)\n\n# Get centroids of clusters\nmean_df = dft.groupby("Topic").mean().reset_index()\nmean_df.Topic = mean_df.Topic.astype(int)\nmean_df = mean_df.sort_values("Topic")\

In [ ]:
"""
import seaborn as sns
from matplotlib import pyplot as plt
from adjustText import adjust_text
import matplotlib.patheffects as pe
import textwrap

fig = plt.figure(figsize=(8, 8))
#sns.color_palette("Paired")
sns.scatterplot(data=df, x='x', y='y', hue = df['Topic'], alpha=0.4, sizes=(0.4, 10), size="Length")


# Annotate top 50 topics
texts, xs, ys = [], [], []
for row in mean_df.iterrows():
  topic = row[1]["Topic"]
  name = textwrap.fill(topic_model.custom_labels_[int(topic)+1], 20)

  if int(topic) <= 50:
    xs.append(row[1]["x"])
    ys.append(row[1]["y"])
    texts.append(plt.text(row[1]["x"], row[1]["y"], name, size=8, ha="center", color=color_key[int(topic)],
                          path_effects=[pe.withStroke(linewidth=0.5, foreground="black")]
                          ))

# Adjust annotations such that they do not overlap
adjust_text(texts, x=xs, y=ys, time_lim=1, force_text=(0.01, 0.02), force_static=(0.01, 0.02), force_pull=(0.5, 0.5))

plt.axis('off')

plt.legend('', frameon=False)
plt.show()
"""

'\nimport seaborn as sns\nfrom matplotlib import pyplot as plt\nfrom adjustText import adjust_text\nimport matplotlib.patheffects as pe\nimport textwrap\n\nfig = plt.figure(figsize=(8, 8))\n#sns.color_palette("Paired")\nsns.scatterplot(data=df, x=\'x\', y=\'y\', hue = df[\'Topic\'], alpha=0.4, sizes=(0.4, 10), size="Length")\n\n\n# Annotate top 50 topics\ntexts, xs, ys = [], [], []\nfor row in mean_df.iterrows():\n  topic = row[1]["Topic"]\n  name = textwrap.fill(topic_model.custom_labels_[int(topic)+1], 20)\n\n  if int(topic) <= 50:\n    xs.append(row[1]["x"])\n    ys.append(row[1]["y"])\n    texts.append(plt.text(row[1]["x"], row[1]["y"], name, size=8, ha="center", color=color_key[int(topic)],\n                          path_effects=[pe.withStroke(linewidth=0.5, foreground="black")]\n                          ))\n\n# Adjust annotations such that they do not overlap\nadjust_text(texts, x=xs, y=ys, time_lim=1, force_text=(0.01, 0.02), force_static=(0.01, 0.02), force_pull=(0.5, 0.5))\n

# Backup Saving

In [ ]:
# To save: model, reduced embeddings, representative docs
#!pip install safetensors

# Sentence Transformer Finetuning
https://huggingface.co/blog/how-to-train-sentence-transformers

In [ ]:
import os, pickle

In [ ]:
path = 'drive//MyDrive//Diploma//'

In [ ]:
with open(os.path.join(path, 'combinations_for_bert.pickle'), 'rb') as file:
    # Deserialize the list and assign it to a variable
    triplets_list = pickle.load(file)
# Print the loaded list

In [ ]:
from sentence_transformers import SentenceTransformer, models

## Step 1: use an existing language model
word_embedding_model_msmarco = models.Transformer('DiTy/bi-encoder-russian-msmarco')
word_embedding_model_inkoz = models.Transformer('inkoziev/sbert_synonymy')

## Step 2: use a pool function over the token embeddings
pooling_model_msmarco = models.Pooling(word_embedding_model_msmarco.get_word_embedding_dimension())
pooling_model_inkoz = models.Pooling(word_embedding_model_inkoz.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
new_model_msmarco = SentenceTransformer(modules=[word_embedding_model_msmarco, pooling_model_msmarco])
new_model_inkoz = SentenceTransformer(modules=[word_embedding_model_inkoz, pooling_model_inkoz])


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authenticatio

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/117M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/410 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
#abstracts_with_punct = df['fr_abbr_with_punct']

In [ ]:
def get_zno_text(doc_id, text_column = 'fr_abbr_with_punct'):
  return df.loc[doc_id][text_column]

In [ ]:
train_df = pd.DataFrame(data = triplets_list, columns=['query_id', 'pos_id', 'neg_id'])

In [4]:
#df.head(1)

In [ ]:
df.set_index('req_id', inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41562 entries, ЗНО0167251285 to ЗНО0341835202
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   req_header              41562 non-null  object        
 1   req_details             41470 non-null  object        
 2   reg_date                41562 non-null  datetime64[ns]
 3   fr_mod_lem              41562 non-null  object        
 4   fr_abbr_with_punct      41562 non-null  object        
 5   fr_mod_lem_swa          41562 non-null  object        
 6   fr_abbr_with_punct_swa  41562 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 2.5+ MB


In [5]:
#df.head(1)

In [ ]:
train_df = train_df.merge(df[['fr_abbr_with_punct']], right_index=True, left_on='query_id').rename(columns={'fr_abbr_with_punct': 'query'})
train_df = train_df.merge(df[['fr_abbr_with_punct']], right_index=True, left_on='pos_id').rename(columns={'fr_abbr_with_punct': 'pos'})
train_df = train_df.merge(df[['fr_abbr_with_punct']], right_index=True, left_on='neg_id').rename(columns={'fr_abbr_with_punct': 'neg'})


#train_df = train_df.merge(df[['full_request_mod']], right_index=True, left_on='query_id').rename(columns={'full_request_mod': 'query'})
#train_df = train_df.merge(df[['full_request_mod']], right_index=True, left_on='pos_id').rename(columns={'full_request_mod': 'pos'})
#train_df = train_df.merge(df[['full_request_mod']], right_index=True, left_on='neg_id').rename(columns={'full_request_mod': 'neg'})

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26336 entries, 0 to 26450
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   query_id  26336 non-null  object
 1   pos_id    26336 non-null  object
 2   neg_id    26336 non-null  object
 3   query     26336 non-null  object
 4   pos       26336 non-null  object
 5   neg       26336 non-null  object
dtypes: object(6)
memory usage: 1.4+ MB


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5268 entries, 25647 to 9777
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   query_id  5268 non-null   object
 1   pos_id    5268 non-null   object
 2   neg_id    5268 non-null   object
 3   query     5268 non-null   object
 4   pos       5268 non-null   object
 5   neg       5268 non-null   object
dtypes: object(6)
memory usage: 288.1+ KB


In [ ]:
from sentence_transformers import InputExample

train_examples = []
n_examples = len(triplets_list)

for i,row in X_train.iterrows() :
  train_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)

### MsMarco model new (with punctuation, no stop words)

In [ ]:
from sentence_transformers import losses

train_loss_msmarco = losses.TripletLoss(model=new_model_msmarco)

In [ ]:
new_model_msmarco.fit(train_objectives=[(train_dataloader, train_loss_msmarco)], epochs=5)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2634 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2634 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2634 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2634 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2634 [00:00<?, ?it/s]

In [ ]:
path = 'drive//MyDrive//Diploma//'
fname = path+'new_model_msmarco'
new_model_msmarco.save(fname)

GOTO1

In [ ]:
from sentence_transformers import SentenceTransformer, models
new_model_msmarco = SentenceTransformer('drive//MyDrive//Diploma//new_model_msmarco')

In [ ]:
from sentence_transformers import SentenceTransformer, util

def get_sim(model, text1, text2):
  v1 = model.encode(text1)
  v2 = model.encode(text2)
  return util.cos_sim(a=v1, b=v2).item()


def check_accuracy(model, test_df):
  pos_sim = []
  neg_sim = []
  check_list = []
  for i,row in test_df.iterrows() :
    s1 = get_sim(model, row['query'], row['pos'])
    s2 = get_sim(model, row['query'], row['neg'])
    pos_sim.append(s1)
    neg_sim.append(s2)
    check_list.append(s1 > s2)

  return pos_sim, neg_sim, check_list

In [ ]:
X_test_msmarco = X_test.copy()

In [ ]:
orignal_model_msmarco = SentenceTransformer('DiTy/bi-encoder-russian-msmarco')
#orignal_model = SentenceTransformer('inkoziev/sbert_synonymy')

#test_df = train_df.sample(frac=0.2)
pos_sim, neg_sim, check_list = check_accuracy(orignal_model_msmarco, X_test_msmarco)

X_test_msmarco['orig_pos_sim'] = pos_sim
X_test_msmarco['orig_neg_sim'] = neg_sim
X_test_msmarco['orig_check_list'] = check_list

pos_sim, neg_sim, check_list = check_accuracy(new_model_msmarco, X_test)

X_test_msmarco['trained_pos_sim'] = pos_sim
X_test_msmarco['trained_neg_sim'] = neg_sim
X_test_msmarco['trained_check_list'] = check_list


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
print('before train mean pos sim', np.mean(X_test_msmarco['orig_pos_sim']))
print('before train mean neg sim', np.mean(X_test_msmarco['orig_neg_sim']))
print('after train mean pos sim', np.mean(X_test_msmarco['trained_pos_sim']))
print('after train mean neg sim', np.mean(X_test_msmarco['trained_neg_sim']))



before train mean pos sim 0.6646386373135061
before train mean neg sim 0.409243622737512
after train mean pos sim 0.9539862811712487
after train mean neg sim -0.054976342811041294


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41562 entries, ЗНО0167251285 to ЗНО0341835202
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   req_header              41562 non-null  object        
 1   req_details             41470 non-null  object        
 2   reg_date                41562 non-null  datetime64[ns]
 3   fr_mod_lem              41562 non-null  object        
 4   fr_abbr_with_punct      41562 non-null  object        
 5   fr_mod_lem_swa          41562 non-null  object        
 6   fr_abbr_with_punct_swa  41562 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 2.5+ MB


In [ ]:
df.dropna(inplace=True)

In [ ]:
#abstracts_with_punct = df['fr_abbr_with_punct']

In [ ]:
embeddings_new_msmarco = new_model_msmarco.encode(abstracts_with_punct, show_progress_bar=True)

Batches:   0%|          | 0/1299 [00:00<?, ?it/s]

In [ ]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings_msmarco_after_train = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings_new_msmarco)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration
# KeyBERT
keybert_msmarco_after_train = KeyBERTInspired()

# MMR
mmr_msmarco_after_train = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Llama 2
#llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model_msmarco_after_train = {
    "KeyBERT": keybert_msmarco_after_train,
 #   "Llama2": llama2,
    "MMR": mmr_msmarco_after_train,
}

In [ ]:
from bertopic import BERTopic

topic_model_msmarco_after_train = BERTopic(

  # Sub-models
  embedding_model = new_model_msmarco,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model_msmarco_after_train,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics_msmarco_after_train, probs_msmarco_after_train = topic_model_msmarco_after_train.fit_transform(abstracts_with_punct, embeddings_new_msmarco)

2024-05-28 18:32:00,605 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-28 18:33:01,541 - BERTopic - Dimensionality - Completed ✓
2024-05-28 18:33:01,544 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
2024-05-28 18:33:07,858 - BERTopic - Cluster - Completed ✓
2024-05-28 18:33:07,870 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-28 18:33:13,006 - BERTopic - Representation - Completed ✓


In [6]:
#topic_model_msmarco_after_train.get_topic_info()

In [7]:
#topic_model_msmarco_after_train.visualize_barchart(top_n_topics = 12, n_words = 7)

In [23]:
#topic_model_msmarco_after_train.get_topic(0, full=True)

In [8]:
#topic_model_msmarco_after_train.visualize_documents(titles, reduced_embeddings=reduced_embeddings_msmarco_after_train, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [ ]:
X_test_msmarco.to_csv(os.path.join(path,"X_test_msmarco_after_train_2805.csv"))

In [ ]:
topic_model_msmarco_after_train.get_topic

<bound method BERTopic.get_topic of <bertopic._bertopic.BERTopic object at 0x7eeaa48aafe0>>

In [ ]:
tp_df = topic_model_msmarco_after_train.get_topic_info()  # dataframe with topics
#topics = pd.Series(topic_model_msmarco_after_train.topics_, index = df.index, name = 'topic_id')  # request - topic pairs
#results = df.join(topics).merge(tp_df[['Topic','Representation','MMR']], left_on = 'topic_id', right_on = 'Topic')
#results.to_csv("drive//MyDrive//bertopic_requests_topic_msmarco_after_train_no_stop_words.csv")
tp_df.to_csv("drive//MyDrive//bertopic_topics_msmarco_after_train_no_stop_words.csv")

## Lets try other embeddings with stop words

In [ ]:
from sentence_transformers import SentenceTransformer, models

## Step 1: use an existing language model
word_embedding_model_msmarco_swa = models.Transformer('DiTy/bi-encoder-russian-msmarco')
#word_embedding_model_inkoz_swa = models.Transformer('inkoziev/sbert_synonymy')

## Step 2: use a pool function over the token embeddings
pooling_model_msmarco = models.Pooling(word_embedding_model_msmarco_swa.get_word_embedding_dimension())
#pooling_model_inkoz = models.Pooling(word_embedding_model_inkoz.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
new_model_msmarco_swa = SentenceTransformer(modules=[word_embedding_model_msmarco_swa, pooling_model_msmarco])
#new_model_inkoz = SentenceTransformer(modules=[word_embedding_model_inkoz, pooling_model_inkoz])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
#df.head(1)

In [ ]:
#abstracts_with_punct_swa = df['fr_abbr_with_punct_swa']

In [ ]:
def get_zno_text(doc_id, text_column = 'abstracts_with_punct_swa'):
  return df.loc[doc_id][text_column]

In [ ]:
train_df = pd.DataFrame(data = triplets_list, columns=['query_id', 'pos_id', 'neg_id'])

In [10]:
#df.head(1)

In [ ]:
df.set_index('req_id', inplace=True)

KeyError: "None of ['req_id'] are in the columns"

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41562 entries, ЗНО0167251285 to ЗНО0341835202
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   req_header              41562 non-null  object        
 1   req_details             41470 non-null  object        
 2   reg_date                41562 non-null  datetime64[ns]
 3   fr_mod_lem              41562 non-null  object        
 4   fr_abbr_with_punct      41562 non-null  object        
 5   fr_mod_lem_swa          41562 non-null  object        
 6   fr_abbr_with_punct_swa  41562 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 2.5+ MB


In [ ]:
df.dropna(inplace=True)

In [11]:
#df.head(1)

GOTO3

In [ ]:
train_df = train_df.merge(df[['fr_abbr_with_punct_swa']], right_index=True, left_on='query_id').rename(columns={'fr_abbr_with_punct_swa': 'query'})
train_df = train_df.merge(df[['fr_abbr_with_punct_swa']], right_index=True, left_on='pos_id').rename(columns={'fr_abbr_with_punct_swa': 'pos'})
train_df = train_df.merge(df[['fr_abbr_with_punct_swa']], right_index=True, left_on='neg_id').rename(columns={'fr_abbr_with_punct_swa': 'neg'})


#train_df = train_df.merge(df[['full_request_mod']], right_index=True, left_on='query_id').rename(columns={'full_request_mod': 'query'})
#train_df = train_df.merge(df[['full_request_mod']], right_index=True, left_on='pos_id').rename(columns={'full_request_mod': 'pos'})
#train_df = train_df.merge(df[['full_request_mod']], right_index=True, left_on='neg_id').rename(columns={'full_request_mod': 'neg'})

In [12]:
#train_df.head(1)

GOTO2

In [ ]:
check_words = ['ru', 'sbrf', 'tvlod', 'pplad', 'база', 'pangolin']
checks_frequency = defaultdict(int)
def checkword(text):
    text2list = text.split(' ')
    for token in text2list:
        if token in check_words:
            checks_frequency[token]+=1
checks_frequency = defaultdict(int)
checks_frequency
train_df['pos'].apply(checkword)
checks_frequency

In [ ]:
checks_frequency = defaultdict(int)
checks_frequency

defaultdict(int, {})

In [ ]:
train_df['pos'].apply(checkword)
checks_frequency

defaultdict(int, {})

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5268 entries, 25647 to 9777
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   query_id  5268 non-null   object
 1   pos_id    5268 non-null   object
 2   neg_id    5268 non-null   object
 3   query     5268 non-null   object
 4   pos       5268 non-null   object
 5   neg       5268 non-null   object
dtypes: object(6)
memory usage: 288.1+ KB


In [ ]:
from sentence_transformers import InputExample

train_examples = []
n_examples = len(triplets_list)

for i,row in X_train.iterrows() :
  train_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)

### MsMarco model new swa

In [ ]:
from sentence_transformers import losses

train_loss_msmarco_swa = losses.TripletLoss(model=new_model_msmarco_swa)

In [ ]:
new_model_msmarco_swa.fit(train_objectives=[(train_dataloader, train_loss_msmarco_swa)], epochs=5)
#new_model_msmarco.fit(train_objectives=[(train_dataloader, train_loss_msmarco)], epochs=5)

Step,Training Loss
500,2.520300
1000,0.850900
1500,0.307800
2000,0.124700
2500,0.052000
3000,0.036600
3500,0.018300
4000,0.005500
4500,0.006200
5000,0.005800


In [ ]:
path = 'drive//MyDrive//Diploma//'
fname = path+'new_model_msmarco_swa'
new_model_msmarco_swa.save(fname)

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

In [ ]:
from sentence_transformers import SentenceTransformer, util

def get_sim(model, text1, text2):
  v1 = model.encode(text1)
  v2 = model.encode(text2)
  return util.cos_sim(a=v1, b=v2).item()


def check_accuracy(model, test_df):
  pos_sim = []
  neg_sim = []
  check_list = []
  for i,row in test_df.iterrows() :
    s1 = get_sim(model, row['query'], row['pos'])
    s2 = get_sim(model, row['query'], row['neg'])
    pos_sim.append(s1)
    neg_sim.append(s2)
    check_list.append(s1 > s2)

  return pos_sim, neg_sim, check_list

In [ ]:
X_test_msmarco_swa = X_test.copy()

In [ ]:
orignal_model_msmarco = SentenceTransformer('DiTy/bi-encoder-russian-msmarco')
#orignal_model = SentenceTransformer('inkoziev/sbert_synonymy')

#test_df = train_df.sample(frac=0.2)
pos_sim, neg_sim, check_list = check_accuracy(orignal_model_msmarco, X_test_msmarco_swa)

X_test_msmarco_swa['orig_pos_sim'] = pos_sim
X_test_msmarco_swa['orig_neg_sim'] = neg_sim
X_test_msmarco_swa['orig_check_list'] = check_list

pos_sim, neg_sim, check_list = check_accuracy(new_model_msmarco_swa, X_test_msmarco_swa)

X_test_msmarco_swa['trained_pos_sim'] = pos_sim
X_test_msmarco_swa['trained_neg_sim'] = neg_sim
X_test_msmarco_swa['trained_check_list'] = check_list


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
np.mean(pos_sim)

0.9453243328061807

In [ ]:
np.mean(neg_sim)

-0.05168285905802793

In [ ]:
X_test_msmarco_swa.orig_neg_sim.mean()

0.3530083441884841

In [ ]:
X_test_msmarco_swa.trained_neg_sim.mean()

-0.05168285905802793

In [ ]:
df.dropna(inplace=True)

In [ ]:
#abstracts_with_punct = df['fr_abbr_with_punct']

In [ ]:
embeddings_new_msmarco_swa = new_model_msmarco_swa.encode(abstracts_with_punct_swa, show_progress_bar=True)

Batches:   0%|          | 0/1299 [00:00<?, ?it/s]

In [ ]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings_msmarco_after_train_swa = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings_new_msmarco_swa)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration
# KeyBERT
keybert_msmarco_after_train_swa = KeyBERTInspired()

# MMR
mmr_msmarco_after_train_swa = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Llama 2
#llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model_msmarco_after_train_swa = {
    "KeyBERT": keybert_msmarco_after_train_swa,
 #   "Llama2": llama2,
    "MMR": mmr_msmarco_after_train_swa,
}

In [ ]:
from bertopic import BERTopic

topic_model_msmarco_after_train_swa = BERTopic(

  # Sub-models
  embedding_model = new_model_msmarco_swa,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model_msmarco_after_train_swa,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics_msmarco_after_train_swa, probs_msmarco_after_train_swa = topic_model_msmarco_after_train_swa.fit_transform(abstracts_with_punct_swa, embeddings_new_msmarco_swa)

2024-05-29 04:28:30,521 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-29 04:30:06,265 - BERTopic - Dimensionality - Completed ✓
2024-05-29 04:30:06,269 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
2024-05-29 04:30:12,180 - BERTopic - Cluster - Completed ✓
2024-05-29 04:30:12,193 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-29 04:30:16,217 - BERTopic - Representation - Completed ✓


In [13]:
#topic_model_msmarco_after_train_swa.get_topic_info()

In [14]:
#topic_model_msmarco_after_train_swa.visualize_barchart(top_n_topics = 14, n_words = 7)

In [15]:
#topic_model_msmarco_after_train_swa.get_topic(0, full=True)

In [16]:
#topic_model_msmarco_after_train_swa.visualize_documents(titles, reduced_embeddings=reduced_embeddings_msmarco_after_train_swa, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [ ]:
X_test_msmarco_swa.to_csv(os.path.join(path,"X_test2905_swa.csv"))

In [ ]:
tp_df = topic_model_msmarco_after_train_swa.get_topic_info()  # dataframe with topics
#topics = pd.Series(topic_model_msmarco_after_train_swa.topics_, index = df.index, name = 'topic_id')  # request - topic pairs
#results = df.join(topics).merge(tp_df[['Topic','Representation','MMR']], left_on = 'topic_id', right_on = 'Topic')
#results.to_csv("drive//MyDrive//bertopic_requests_topic_msmarco_swa_1.csv")
tp_df.to_csv("drive//MyDrive//bertopic_topics_msmarco_swa_2905.csv")